In [6]:
run_in_notebook = False

In [2]:
import sys
sys.path.insert(0, '/home/jovyan/braindecode/')
#sys.path.insert(0, '/home/jovyan/mne-python/')

from decode_tueg import decode_tueg

In [3]:
import time
import logging

from sklearn.model_selection import ParameterGrid, ParameterSampler
import pandas as pd

In [4]:
import os
import glob
import datetime
from typing import List
from functools import reduce

import json
from kfp import Client
from kfp.compiler import Compiler
from kfp.components import (
    create_component_from_func, load_component_from_text, OutputPath)
from kfp.dsl import (
    PipelineParam, PipelineVolume, pipeline as dsl_pipeline, RUN_ID_PLACEHOLDER)
from kubernetes.client.models.v1_toleration import V1Toleration
from kubernetes.client import V1EmptyDirVolumeSource, V1Volume
client = Client()

In [5]:
import torch
torch.version.cuda

'11.7'

In [7]:
torch.cuda.is_available()

True

In [8]:
pipe_suffix = 'decode_age'

In [9]:
job_definition = """
name: TUAB age decoding
inputs:
- name: augment
  type: String
- name: batch_size
  type: Integer
- name: condition
  type: String
- name: data_path
  type: String
- name: date
  type: String
- name: debug
  type: Integer
- name: fast_mode
  type: Integer
- name: final_eval
  type: Integer
- name: intuitive_training_scores
  type: Integer
- name: loss
  type: String
- name: max_age
  type: Integer
- name: min_age
  type: Integer
- name: model_name
  type: String
- name: n_epochs
  type: Integer
- name: n_jobs
  type: Integer
- name: n_restarts
  type: Integer
- name: n_train_recordings
  type: Integer
- name: out_dir
  type: String
- name: preload
  type: Integer
- name: seed
  type: Integer
- name: shuffle_data_before_split
  type: Integer
- name: squash_outs
  type: Integer
- name: standardize_data
  type: Integer
- name: standardize_targets
  type: Integer
- name: subset
  type: String
- name: target_name
  type: String
- name: tmax
  type: Integer
- name: tmin
  type: Integer
- name: valid_set_i
  type: Integer
- name: window_size_samples
  type: Integer
- name: run_name
  type: String
  
  
implementation:
    container:
        # This is using a development version of the job executor, which might 
        # unexpectedly change,
        # You probably want to got to https://cr.nail.science and pick the latest
        # stable tag, e.g. v1.0.0, to use in your experiments.
        image: cr.nail.science/ml-containers/python-job:git-7f92f40
        command: ['/init', 'python', 'tueg_age_decoding/decode_tueg.py']
        args: [
            '--augment', {inputValue: augment},
            '--batch-size', {inputValue: batch_size},
            '--condition', {inputValue: condition},
            '--data-path', {inputValue: data_path},
            '--date', {inputValue: date},
            '--debug', {inputValue: debug},
            '--fast-mode', {inputValue: fast_mode},
            '--final-eval', {inputValue: final_eval},
            '--intuitive-training-scores', {inputValue: intuitive_training_scores},
            '--loss', {inputValue: loss},
            '--max-age', {inputValue: max_age},
            '--min-age', {inputValue: min_age},
            '--model-name', {inputValue: model_name},
            '--n-epochs', {inputValue: n_epochs},
            '--n-jobs', {inputValue: n_jobs},
            '--n-restarts', {inputValue: n_restarts},
            '--n-train-recordings', {inputValue: n_train_recordings},
            '--out-dir', {inputValue: out_dir},
            '--preload', {inputValue: preload},
            '--seed', {inputValue: seed},
            '--shuffle-data-before-split', {inputValue: shuffle_data_before_split},
            '--squash-outs', {inputValue: squash_outs},
            '--standardize-data', {inputValue: standardize_data},
            '--standardize-targets', {inputValue: standardize_targets},
            '--subset', {inputValue: subset},
            '--target-name', {inputValue: target_name},
            '--tmax', {inputValue: tmax},
            '--tmin', {inputValue: tmin},
            '--valid-set-i', {inputValue: valid_set_i},
            '--window-size-samples', {inputValue: window_size_samples},
            '--run-name', {inputValue: run_name},
        ]
"""

In [10]:
f"{client.get_user_namespace()}-{pipe_suffix}"

'kf-gemein-lukas-decode_age'

In [11]:
@dsl_pipeline(name=f"{client.get_user_namespace()}-{pipe_suffix}")
def demo_pipeline(
    augment:str,
    batch_size:int,
    condition:str,
    data_path:str,
    date:str,
    debug:int,
    fast_mode:int,
    final_eval:int,
    intuitive_training_scores:int,
    loss:str,
    max_age:int,
    min_age:int,
    model_name:str,
    n_epochs:int,
    n_jobs:int,
    n_restarts:int,
    n_train_recordings:int,
    out_dir:str,
    preload:int,
    seed:int,
    shuffle_data_before_split:int,
    squash_outs:int,
    standardize_data:int,
    standardize_targets:int,
    subset:str,
    target_name:str,
    tmax:int,
    tmin:int,
    valid_set_i:int,
    window_size_samples:int,
    run_id:str=RUN_ID_PLACEHOLDER,
):
    workspace = PipelineVolume(pvc=f'workspace-tueg')
    mne_data = PipelineVolume(pvc='mne-data')
    shared_volume = PipelineVolume(volume=V1Volume(
        name="shm",
        empty_dir=V1EmptyDirVolumeSource(medium='Memory')))
    
    run_name = f"decode_age"
    # on disk, preprocessed TUHAbnormal is 17GB
    mem_lim = '20Gi'
    mem_req = '20Gi'

    (load_component_from_text(job_definition)(
        augment=augment,
        batch_size=batch_size,
        condition=condition,
        data_path=data_path,
        date=date,
        debug=debug,
        fast_mode=fast_mode,
        final_eval=final_eval,
        intuitive_training_scores=intuitive_training_scores,
        loss=loss,
        max_age=max_age,
        min_age=min_age,
        model_name=model_name,
        n_epochs=n_epochs,
        n_jobs=n_jobs,
        n_restarts=n_restarts,
        n_train_recordings=n_train_recordings,
        out_dir=out_dir,
        preload=preload,
        seed=seed,
        shuffle_data_before_split=shuffle_data_before_split,
        squash_outs=squash_outs,
        standardize_data=standardize_data,
        standardize_targets=standardize_targets,
        subset=subset,
        target_name=target_name,
        tmax=tmax,
        tmin=tmin,
        valid_set_i=valid_set_i,
        window_size_samples=window_size_samples,
        run_name=run_name,
    )
    .add_pvolumes({
        '/work': workspace,
        '/home/jovyan/mne_data': mne_data,
        '/dev/shm': shared_volume,
    })
    # If you do not add this, your job will download all the packages on every run. 
    # Which will be veery slow and congest our network ...
    .add_pod_label('shared-package-cache', 'true')
    # Request the required resources
    .set_cpu_limit('4')
    .set_cpu_request('4')
    .set_memory_limit(mem_lim)
    .set_memory_request(mem_req)
    # This is needed so your job can be scheduled on GPU nodes 
    .add_toleration(
        V1Toleration(key='nvidia.com/gpu', operator='Exists', effect='NoSchedule'))
    .set_gpu_limit('1')
    )

In [12]:
yml_file = '/home/jovyan/workflow.yaml'
pipeline_name = f"{client.get_user_namespace()}-{pipe_suffix}"

In [13]:
pipeline_name

'kf-gemein-lukas-decode_age'

In [14]:
Compiler().compile(demo_pipeline, yml_file)

In [15]:
pipeline_id = client.get_pipeline_id(pipeline_name)
if pipeline_id is not None:
    pipeline = client.get_pipeline(pipeline_id)
    pipline = client.upload_pipeline_version(
        yml_file, pipeline_id=pipeline_id, 
        pipeline_version_name=datetime.datetime.now().isoformat())
else:
    pipeline = client.upload_pipeline(yml_file, pipeline_name=pipeline_name)

In [16]:
pipeline_name

'kf-gemein-lukas-decode_age'

In [17]:
try:
    experiment = client.get_experiment(experiment_name=pipe_suffix)
except ValueError:
    experiment = client.create_experiment(pipe_suffix)

In [18]:
base_dir = '/home/jovyan/' if run_in_notebook else '/work/'

In [19]:
exp_date = datetime.datetime.now().isoformat()

In [20]:
exp_date

'2023-02-01T10:21:55.067691'

In [21]:
param_grid = ParameterGrid({
    'model_name': ['tcn'],  # 'shallow', 'deep', 'tcn'
    'subset': ['normal'],  # 'normal', 'abnormal', 'mixed'
    'target_name': ['age'],  # age, gender, pathological, age_clf

    'valid_set_i': [0,1,2,3,4],  # 0, 1, 2, 3, 4
    'n_epochs': [35],  # 35, 105, 210
    'n_restarts': [0],  # 0, 2, 5
    'augment': ['dropout'],  # dropout, flipfb, fliplr, noise, mask, reverse, shuffle, sign, random, identity, '0'
    'fast_mode': [1],
    'loss': ['mae'],  # mse, mae, log_cosh, huber, nll

    'n_train_recordings': [-1],  # -1: None
    'tmax': [-1],  # 4*60done, 6*60done, 11*60done, -1,  00# -1: None
    'min_age': [-1],
    'max_age': [-1],
    'data_path': ['/home/jovyan/mne_data/TUH_PRE/tuh_eeg_abnormal/v2.0.0/edf/'],
    'squash_outs': [1],  # force output to be in [0, 1] through sigmoid

    'final_eval': [0],
    'debug': [1],
    'seed': [20230111],  # last 20221116, default 20220429  # 20230229: "optimized" train eval split on tuh abnormal
    'date': [exp_date],  # sometimes, need to restart some of the cv runs, due to cluster failure. do not reset exp date then
    'intuitive_training_scores': [1],  # 1: add slow callbacks that track age decodnig loss intuitively as mae
    'out_dir': [os.path.join(base_dir, 'experiments')],
    'n_jobs': [4],  # faster than 1, 3, and 4 on tmax=2*60, n_recordings=-1, subset=normal, n_epochs=5, preload=0
    'preload': [0],

    'batch_size': [64],  # 64. does CroppedTrialEpochStoring increase GPU memory consumption? 256 works fine in notebook but fails as pipeline. 128 works with shallow fails with deep
    'tmin': [60],
    'standardize_data': [1],  # TODO: needs to be implemented. sclaing to microvolts is done anyways
    'standardize_targets': [1],
    'window_size_samples': [6000],
    'shuffle_data_before_split': [0],

    # competition stuffs leftover / remove
    # 'data_path': ['/home/jovyan/competition/data/training/'],
    # 'window_size_samples': [1500],  # EC condition is ~40s, EO only ~20s
    'condition': ['all'],  # 'all', 'EC', 'EO', TODO: implement using both, prevent subject leakage in both sets
    # 'out_dir': [os.path.join(base_dir, 'competition/results/')],
})

In [22]:
len(param_grid)

5

In [ ]:
if run_in_notebook:
    for params_i, params in enumerate(param_grid):
        # logging makes the second iteration crash....
        decode_tueg(**params, config=pd.Series(params, name='config'))
else:
#     time.sleep(3)
    runs = []
    for params in param_grid:
        job_name = f"{pipe_suffix}-{params['date']}-{params['seed']}-{params['valid_set_i']}"
        run = client.run_pipeline(    
            experiment_id=experiment.id, 
            pipeline_id=pipeline.id, 
            job_name=job_name,
            params=params,
        )
        runs.append(run)
#         time.sleep(1)

2023-01-31 17:21:40 INFO     
augment                                                                dropout
batch_size                                                                  64
condition                                                                  all
data_path                    /home/jovyan/mne_data/TUH_PRE/tuh_eeg_abnormal...
date                                                2023-01-31T17:21:38.471710
debug                                                                        1
fast_mode                                                                    1
final_eval                                                                   0
intuitive_training_scores                                                    1
loss                                                                       mae
max_age                                                                     -1
min_age                                                                     -1
model_name            

In [ ]:
print("hi")